# Module 1, Section 1: Foundational Concepts

In this section, we'll build a simple customer support system for TechHub, a fictional e-commerce electronics store. 

<div align="center">
    <img src="../../images/db_agent.png">
</div>

We'll start with the basics:
1. How to make LLM calls with messages
2. How to define tools that access external data (our TechHub database)
3. The manual tool calling loop (involved but educational!)

By the end, you'll understand **why** agent frameworks exist - spoiler, they automate the tedious parts we're about to implement manually

## Setup

First, let's load our environment variables (API keys).


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

## 1. Basic LLM Call

Let's start simple - just calling an LLM with a message.

**Note:** The default model is set workshop-wide via WORKSHOP_MODEL in the .env file (see config.py).


In [ ]:
from langchain.chat_models import init_chat_model
from config import DEFAULT_MODEL

print(f"Using model: {DEFAULT_MODEL}")

# Initialize the model (using workshop default)
llm = init_chat_model(DEFAULT_MODEL)

# Simple string input
response = llm.invoke("What is LangChain in under 10 words?")
response.pretty_print()

## 2. Working with Messages

LLMs work with **messages** that have roles:
- `SystemMessage`: Instructions for how the LLM should behave
- `HumanMessage`: User input
- `AIMessage`: Model responses
- `ToolMessage`: Results from tool execution (we'll see this soon!)


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Multi-turn conversation with messages
messages = [
    SystemMessage(
        content="You are a helpful customer support assistant for TechHub, an electronics store."
    ),
    HumanMessage(content="Hello!"),
    AIMessage(content="Hi! How can I help you today?"),
    HumanMessage(content="What's the status of order ORD-2024-0123?"),
]

response = llm.invoke(messages)
response.pretty_print()

**Notice the problem:** The LLM can't actually look up the order! It doesn't have access to our database.

This is where **tools** come in.


## 3. Defining Tools

<div align="center">
    <img src="../../images/db_tools.png">
</div>

Tools give LLMs the ability to interact with external systems. Let's create four tools that query our [TechHub database](../../data/structured/techhub_schema.png).

The `@tool` decorator from `langchain` automatically:
- Extracts the function signature for the LLM
- Uses the docstring as the tool description
- Handles the input/output formatting

In [ ]:
from pathlib import Path
from langchain.tools import tool
from langchain_community.utilities import SQLDatabase

# instaniate the database connection
DB_PATH = Path("../../data/structured/techhub.db")
db = SQLDatabase.from_uri(f"sqlite:///{DB_PATH}")


# helper function
def extract_values(result):
    """Convert SQLDatabase query results (list of dicts) to list of tuples (values only)."""
    return [tuple(row.values()) for row in result]


@tool
def get_order_status(order_id: str) -> str:
    """Get status, dates, and tracking information for a specific order.

    Note: For order total, calculate from items using get_order_item_price().

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with order status, dates, and tracking number.
    """
    result = db._execute(
        f"""
        SELECT order_id, order_date, status, shipped_date, tracking_number
        FROM orders
        WHERE order_id = '{order_id}'
    """
    )
    result = extract_values(result)

    if not result:
        return f"Order {order_id} not found."

    order_id, order_date, status, shipped_date, tracking_number = result[0]

    response = f"Order {order_id}:\n"
    response += f"- Status: {status}\n"
    response += f"- Order Date: {order_date}\n"

    if shipped_date:
        response += f"- Shipped Date: {shipped_date}\n"
    if tracking_number:
        response += f"- Tracking Number: {tracking_number}\n"

    return response


@tool
def get_order_items(order_id: str) -> str:
    """Get list of items in a specific order with product IDs and quantities.

    Note: For pricing, use get_order_item_price() for historical price paid, or get_product_info() for current price.

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with product IDs and quantities (no prices).
    """
    result = db._execute(
        f"""
        SELECT product_id, quantity
        FROM order_items
        WHERE order_id = '{order_id}'
    """
    )
    result = extract_values(result)

    if not result:
        return f"No items found for order {order_id}."

    response = f"Items in order {order_id}:\n"
    for product_id, quantity in result:
        response += f"- Product ID: {product_id}, Quantity: {quantity}\n"

    return response


@tool
def get_product_info(product_identifier: str) -> str:
    """Get product details by product name or product ID.

    Args:
        product_identifier: Product name (e.g., "MacBook Air") or product ID (e.g., "TECH-LAP-001")

    Returns:
        Formatted string with product name, category, price, and stock status.
    """
    # Try exact ID match first
    result = db._execute(
        f"""
        SELECT product_id, name, category, price, in_stock
        FROM products
        WHERE product_id = '{product_identifier}'
    """
    )
    result = extract_values(result)

    # If no exact match, try fuzzy name search
    if not result:
        result = db._execute(
            f"""
            SELECT product_id, name, category, price, in_stock
            FROM products
            WHERE name LIKE '%{product_identifier}%'
            LIMIT 1
        """
        )
        result = extract_values(result)

    if not result:
        return f"Product '{product_identifier}' not found."

    product_id, name, category, price, in_stock = result[0]
    stock_status = "In Stock" if in_stock else "Out of Stock"

    return f"{name} ({product_id})\n- Category: {category}\n- Price: ${price:.2f}\n- Status: {stock_status}"


@tool
def get_order_item_price(order_id: str, product_id: str) -> str:
    """Get the historical price paid for a specific item in an order.

    Use this to get the actual price the customer paid at time of purchase,
    which may differ from the current retail price from get_product_info().

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")
        product_id: The product ID (e.g., "TECH-LAP-001")

    Returns:
        Formatted string with historical price per unit.
    """
    result = db._execute(
        f"""
        SELECT price_per_unit, quantity
        FROM order_items
        WHERE order_id = '{order_id}' AND product_id = '{product_id}'
        """
    )
    result = extract_values(result)

    if not result:
        return f"Item {product_id} not found in order {order_id}."

    price, quantity = result[0]
    return f"Historical price for {product_id} in {order_id}: ${price:.2f} per unit (quantity: {quantity})"

Let's test out the `get_order_status` tool

In [ ]:
example_order = "ORD-2024-0127"
result = get_order_status.invoke(example_order)
print(result)

Let's inspect how the `@tool` decorator parses the tool information into a `StructuredTool` object.


In [ ]:
print("\n--- TOOL NAME ---")
print(get_order_status.name)

print("\n--- TOOL DESCRIPTION ---")
print(get_order_status.description)

print("\n--- TOOL ARGUMENTS ---")
print(get_order_status.args)

In [ ]:
get_order_status.tool_call_schema.model_json_schema()

## 4. The Manual Tool Calling Loop

Now let's see how LLMs actually [use tools](https://docs.langchain.com/oss/python/langchain/models#tool-calling)! This happens in stages:

<div align="center">
    <img src="../../images/function-calling.png">
    <br>
    <sub>
        Image Source: <a href="https://www.philschmid.de/gemini-function-calling">Phil Schmid Blog</a>
    </sub>
</div>

Let's see each stage in action!


### Step 0: Bind tools to the model

First, we tell the LLM what tools are available by "binding" them. This ensures that the available tool definitions are included with the model request.


In [ ]:
# Bind tools to the model - this tells the LLM what tools are available
tools = [get_order_status, get_order_items, get_product_info, get_order_item_price]
llm_with_tools = llm.bind_tools(tools)

### Steps 1, 2, 3: Call the LLM with query, it decides to call a tool, and returns the formated tool call object

Now let's give the LLM a query. It will decide which tool to call based on the available tool descriptions and respond with a tool call object including tool arguments.


In [ ]:
# Ask about an order
messages = [
    SystemMessage(content="You are a helpful customer support assistant."),
    HumanMessage(content="What's the status of order ORD-2024-0123?"),
]

# Call the LLM
response = llm_with_tools.invoke(messages)
response.pretty_print()

# Add the AI's tool call to messages
messages.append(response)

### Step 4: Manually execute the tool

The LLM told us _what_ to call and _with what arguments_. Now **we** need to actually execute the tool with the specified arguments.


In [ ]:
# Extract the tool call information
tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]
tool_call_id = tool_call["id"]

print(f"[Executing]\n Tool: {tool_name}\n Args: {tool_args}\n ID: {tool_call_id}", "\n")

# Manually execute the tool
if tool_name == "get_order_status":
    tool_result = get_order_status.invoke(tool_args)
elif tool_name == "get_order_items":
    tool_result = get_order_items.invoke(tool_args)
elif tool_name == "get_product_info":
    tool_result = get_product_info.invoke(tool_args)
elif tool_name == "get_order_item_price":
    tool_result = get_order_item_price.invoke(tool_args)

print(f"[Tool Result]\n {tool_result}")

### Step 5: Pass results back to the LLM

Now we need to send the tool result back to the LLM so it can generate a final answer for the user in natural language. Note that we use a `ToolMessage` type when adding tool result details back to the message history.


In [ ]:
from langchain_core.messages import ToolMessage

# Create a ToolMessage with the result
tool_message = ToolMessage(
    content=tool_result,
    tool_call_id=tool_call["id"],  # Must match the ID from the tool call
)
messages.append(tool_message)
messages

### Step 6: Get the Final Answer

Now the LLM has the tool result and can give a complete answer in natural language to the user.


In [ ]:
# Call the LLM again with the tool result
final_response = llm_with_tools.invoke(messages)

final_response.pretty_print()

### Putting It All Together: The Loop

In real scenarios, the LLM might need to call multiple tools or loop several times. Let's create a function that automates this process:


In [ ]:
def run_agent_loop(user_query: str):
    """Run the complete tool calling loop."""

    messages = [
        SystemMessage(
            content="You are a helpful customer support assistant for TechHub."
        ),
        HumanMessage(content=user_query),
    ]

    print(f"\n{'='*60}")
    print(f"User: {user_query}")
    print(f"{'='*60}\n")

    # Keep looping until we get a final answer
    iteration = 0
    while True:
        iteration += 1

        # Call the LLM
        response = llm_with_tools.invoke(messages)

        # Check if LLM wants to use tools
        if not response.tool_calls:
            # No tools needed - we have the final answer!
            print(f"[Final Answer]\n {response.content}\n")
            break

        # LLM wants to use tools
        print(
            f"[Iteration {iteration}] LLM is calling {len(response.tool_calls)} tool(s)..."
        )
        messages.append(response)

        # Execute each tool
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Execute the tool
            if tool_name == "get_order_status":
                result = get_order_status.invoke(tool_args)
            elif tool_name == "get_order_items":
                result = get_order_items.invoke(tool_args)
            elif tool_name == "get_product_info":
                result = get_product_info.invoke(tool_args)
            elif tool_name == "get_order_item_price":
                result = get_order_item_price.invoke(tool_args)

            arg_str = ", ".join(f"{k}={v!r}" for k, v in tool_args.items())
            print(f"  → {tool_name}({arg_str})")

            # Add tool result to messages
            messages.append(
                ToolMessage(content=str(result), tool_call_id=tool_call["id"])
            )

        print()  # Blank line before next iteration

### Try It Out!

Now let's test our agent loop with different queries:


**Example 1: Simple order lookup**


In [ ]:
run_agent_loop("What's the status of order ORD-2024-0123?")

**Example 2: Product price lookup**


In [ ]:
run_agent_loop("How much does the MacBook Air cost?")

**Example 3: Multiple tools in one query**


In [ ]:
run_agent_loop(
    "What's the status of order ORD-2024-0123, what was in it, and how much did it cost?"
)